# Mapping through time

In [ ]:
import geopandas as gpd
import pandas as pd
#from shapely import wkt

import branca
import folium

## Load all data

In [ ]:
## Lire les chefs lieux et les terrains pour l'habillage
path_to_geojsonfiles = "C:/Travail/Enseignement/Cours_M2_python/2023/data/INSEE/"

#Lire les chefs lieux 
cheflieux_url = (
    path_to_geojsonfiles+"chef-lieux_carto75_2022_2154.geojson"
)

cheflieux = gpd.read_file(cheflieux_url) #, where="INSEE_DEP='79'"
cheflieux.crs #2154
cheflieux = cheflieux.to_crs("EPSG:4326")
cheflieux.head()

#Lire les terrains
terrains = gpd.read_file(path_to_geojsonfiles+"terrains_imhana_2154.geojson")
type(terrains) #geopandas.geodataframe.GeoDataFrame
terrains.crs
terrains = terrains.to_crs("EPSG:4326")
terrains.head()

In [ ]:
communes_URL = path_to_geojsonfiles+'communes_NA2022_4326.geojson'
zones = gpd.read_file(communes_URL)
print(zones.shape) #(4309, 6)

# Supprimer la colonne id, l'index sauvegardé dans le fichier, mais inutile. 

zones.drop(['id'], axis=1, inplace=True)
print(zones.shape) #(4309, 5)


dtype_dict = {'insee_com':'string', 'nom':'string', 'nomepci':'string'} #Il fallait convertir nomepci en string pour faire la jointure avec data
zones = zones.astype(dtype = dtype_dict)
#zones.set_index('insee_com', inplace=True)

zones.head()

In [ ]:
communes_ts = pd.read_csv(path_to_geojsonfiles+'communes_ts_etrangerscommunes.csv')
print(communes_ts.shape)
dtype_dict = {'code':'string'} #,  'annee_dt':'timestamps'
communes_ts = communes_ts.astype(dtype = dtype_dict)
communes_ts.head() #7 colonnes

In [ ]:
import json
styledict_communes_path = path_to_geojsonfiles+'styledict_communes.json'
output = open(styledict_communes_path, "r")
styledict_communes = json.load(output)
styledict_communes

## Do the map

In [ ]:
communes_ts.set_index('code', inplace=True) #60326 rows × 7 columns

In [ ]:
test = zones.join(communes_ts,   how='left', on="insee_com") #Jointure sur la column insee_com	 de zones qui égale l'index de communes_ts (code)
print(test.shape)

test #60326 rows × 11 columns

In [ ]:
#Important pour le slider temporel, spécifier que insee_com est l'index des données de geom 
communes_geom = zones.set_index('insee_com')
communes_geom.head()

In [ ]:
# Look only at the map as background layer
#communes_geom = zones
ax = communes_geom.plot(figsize=(10, 10))
ax

In [ ]:
#https://colorbrewer2.org/#type=diverging&scheme=Spectral&n=6
colormap = branca.colormap.StepColormap(
    vmin=test["petranger"].quantile(0.0),
    vmax=test["petranger"].quantile(1),
    colors=['#4a1486', '#1d91c0','#7fcdbb','#fee08b','#fc8d59', '#d53e4f', '#b2182b'],
    index=[ test["petranger"].quantile(0.10),test["petranger"].quantile(0.30), test["petranger"].quantile(0.50), test["petranger"].quantile(0.70), test["petranger"].quantile(0.90), test["petranger"].quantile(0.95)],
    caption="Part d'étrangers (%)"
)



### Prepare the data (if required ONLY ! )

In [ ]:
## A faire  ssi les colonnes annee_dt, color, opacity n'existent pas dans communes_ts: ajoute des colonnes à communes_ts
communes_ts['annee_dt'] = communes_ts['annee'].apply(lambda x: pd.to_datetime(str(x)+'-01-01', format='%Y-%m-%d'))
communes_ts['annee_dt'] = communes_ts['annee_dt'].astype("int64")
#Division entière par 1000 avec // 1000 : .astype(int) / 10**9
communes_ts['annee_dt'] = communes_ts['annee_dt'] // 10 ** 9
communes_ts['annee_dt'] = communes_ts['annee_dt'].astype("U10") 

# Most important : the color is using the colormap defined for whatever the years
communes_ts['color'] = communes_ts['petranger'].apply(colormap) 
communes_ts['opacity'] = 1

In [ ]:
#Construire le dictionnaire de style dont les clés sont les codes insee des communes puis ensuite un couple (couleur, opacité) indexé par l'année, pour chaque entité

# Voir https://python-visualization.github.io/folium/latest/user_guide/plugins/timeslider_choropleth.html 
import numpy as np
import datetime
 
styledata = {}

for country in communes_ts.index:
    df = communes_ts.query("code=='{}'".format(country))
    df = df.set_index('annee_dt')
    styledata[country] = df[['color', 'opacity']]
#3min de temps de calcul  

styledict_communes = {
    str(country): data.to_dict(orient="index") for country, data in styledata.items()
}


### Use the slider

In [ ]:
#Using a slider

# Tuto : https://python-visualization.github.io/folium/latest/user_guide/plugins/timeslider_choropleth.html
# Voir aussi https://github.com/germaniuss/folium-timesliderchoropleth pour ajouter des tooltips
# https://gis.stackexchange.com/questions/442764/folium-timesliderchoropleth-with-tooltip 

import branca

style_tooltip_txt = """
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    
##Minimap
from folium.plugins import MiniMap

min_lon, max_lon = -2, +2
min_lat, max_lat = 43, 47

m = folium.Map(location=(45.00, 0.15605), zoom_start=8, tiles="cartodb positron", width=1000, height=1200, 
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_lon,
        max_lon=max_lon) #45.64844

## Titre : https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map
map_title = "Répartition communale de la population étrangère en Nouvelle-Aquitaine, sur un siècle [1926-2022]"
source_text = "Données : INSEE [producteur], <a href='https://www.unehistoireduconflitpolitique.fr/telecharger.html'>Cagé-Piketty</a> [diffuseur] <br>&copy IGN Admin Express - millesime 2022 <br> Réalisation UMR 7301 Migrinter, C. Plumejeaud, 05-06-2024"
title_html = f'<h4 style="position:absolute;z-index:100000;left:20vh;bottom:95vh;font:bold" >{map_title}</h4>'
source_html = f'<h5 style="position:absolute;z-index:100001;left:70vh;bottom:0vw" >{source_text}</h5>'
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(source_html))

## add markers for chef-lieux 
group_1 = folium.FeatureGroup("chef-lieux").add_to(m)
popup_content = '<table><tr><td>Nom</td><td>{0}</td></tr><tr><td>Statut</td><td>{1}</td></tr><tr><td>Département</td><td>{2}</td></tr><tr><td>Population 2020</td><td>{3}</td></tr></table>'
for index, row in cheflieux.iterrows() :
    #position des markers  : [latitude, longitude]
    folium.Marker(
        location=[row.geometry.geoms[0].coords[0][1], row.geometry.geoms[0].coords[0][0]],
        tooltip=row.NOM_2,
        popup=popup_content.format(row.NOM_2, row.STATUT, row.INSEE_DEP, row.POPULATION),
        icon=folium.Icon(color="green"),
    ).add_to(group_1)
    
group_00 = folium.FeatureGroup("etrangers_animation").add_to(m)
## Les communes, année 1926 à 2018
from folium.plugins import TimeSliderChoropleth

TimeSliderChoropleth(
    communes_geom.to_json(),
    styledict=styledict_communes,
    #init_timestamp=-1,
).add_to(group_00)

group_01 = folium.FeatureGroup("Description des communes, 2022").add_to(m)
#colormap(x["properties"]["petranger"]) if x["properties"]["petranger"] is not None else
## Les communes, année 2022, en transparence
g = folium.GeoJson(
    test.query("annee==2022"),
    style_function=lambda x: {
        "fillColor":  "transparent", 
        "color": "black", 
        "fillOpacity": 1,
        "weight": 0.5,
        "opacity": 0.65
    },
    ## Bulles avec les infos numériques sur les communes
    tooltip=folium.GeoJsonTooltip(
        fields=["nom", "petranger"],
        aliases=["Commune:", "Estimation étrangers en 2022 (%):"],
        localize=True,
        sticky=False,
        labels=True,
        style=style_tooltip_txt,
    )
).add_to(group_01)

## Les terrains
group_2 = folium.FeatureGroup("terrains").add_to(m)
# Courtours bleu : #00FFFFFF, blanc  : FFFFFF
for _, r in terrains.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "#00000000", "color": "#000000"})
    folium.Popup(r["zone_etude"]).add_to(geo_j)
    geo_j.add_to(group_2)

colormap.add_to(m)

MiniMap().add_to(m)
folium.LayerControl().add_to(m)

#4s



In [ ]:
# temps d'affichage 9 s  
m 

In [ ]:
print('Saving the temporal map file...')
m.save('ocarto_commune_petranger_animation_1926-2018.html') #10s